# Нейросети в обработке текста

## Домашнее задание
1. Попробуйте обучить нейронную сеть с применением одномерных сверток для предсказания сентимента сообщений с твитера на примере https://www.kaggle.com/datasets/arkhoshghalb/twitter-sentiment-analysis-hatred-speech

2. Опишите, какой результат вы получили? Что помогло вам улучшить ее точность?

У кого нет возможности работать через каггл (нет верификации), то можете данные взять по ссылке: https://drive.google.com/file/d/1S0hslcWDrwxA5GH4U32mfg85lAVLqkq5/view?usp=sharing

In [155]:
# Context
# The objective of this task is to detect hate speech in tweets. For the sake of simplicity, we say a tweet contains hate 
# speech if it has a racist or sexist sentiment associated with it. So, the task is to classify racist or sexist tweets from 
# other tweets.
# Formally, given a training sample of tweets and labels, where label '1' denotes the tweet is racist/sexist and label '0' 
# denotes the tweet is not racist/sexist, your objective is to predict the labels on the test dataset.

# Контекст
# Цель этой задачи - обнаружить ненавистнические высказывания в твитах. Для простоты мы говорим, что твит содержит 
# ненавистнические высказывания, если с ним связаны расистские или сексистские настроения. Итак, задача состоит в том, чтобы 
# отличить расистские или сексистские твиты от других твитов.
# Формально, учитывая обучающую выборку твитов и меток, где метка "1" означает, что твит является расистским/сексистским, 
# а метка "0" означает, что твит не является расистским /сексистским, ваша цель - предсказать метки в тестовом наборе данных.

In [193]:
import pandas as pd
import numpy as np

In [194]:
path = '\PyTorch-6'
path = ''

In [195]:
df_train = pd.read_csv("train.csv",index_col=0)
df_test = pd.read_csv("test.csv",index_col=0)

In [196]:
df_train

,label,tweet
id,,
1,0,@user when a father is dysfunctional and is s...
2,0,@user @user thanks for #lyft credit i can't us...
3,0,bihday your majesty
4,0,#model i love u take with u all the time in ...
5,0,factsguide: society now #motivation
...,...,...
31958,0,ate @user isz that youuu?ðððððð...
31959,0,to see nina turner on the airwaves trying to...
31960,0,listening to sad songs on a monday morning otw...


In [197]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31962 entries, 1 to 31962
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   31962 non-null  int64 
 1   tweet   31962 non-null  object
dtypes: int64(1), object(1)
memory usage: 749.1+ KB


In [198]:
df_train['label'].value_counts()

0    29720
1     2242
Name: label, dtype: int64

In [199]:
df_test

,tweet
id,
31963,#studiolife #aislife #requires #passion #dedic...
31964,@user #white #supremacists want everyone to s...
31965,safe ways to heal your #acne!! #altwaystohe...
31966,is the hp and the cursed child book up for res...
31967,"3rd #bihday to my amazing, hilarious #nephew..."
...,...
49155,thought factory: left-right polarisation! #tru...
49156,feeling like a mermaid ð #hairflip #neverre...
49157,#hillary #campaigned today in #ohio((omg)) &am...


In [200]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17197 entries, 31963 to 49159
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   17197 non-null  object
dtypes: object(1)
memory usage: 268.7+ KB


In [201]:
from sklearn.model_selection import train_test_split

In [202]:
# Разбиение датасета на train и val

In [203]:
df_train, df_val = train_test_split(df_train, 
                                    test_size=0.2, 
                                    random_state=10, 
                                    stratify=df_train['label'])

df_train.shape, df_val.shape

((25569, 2), (6393, 2))

In [204]:
# Предобработка

In [205]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [206]:
puncts = set(punctuation)
# Не будем очищать текст от апострофов, заменим их потом на пробелы,
# т.к. встроенные в nltk английские стоп-слова и так потом отфильтруют лишнее
puncts = puncts - {"'"}

In [207]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [208]:
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Relict/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Relict/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [209]:
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = ''.join(char for char in txt if char not in puncts) # очистка от пунктуации
    txt = txt.replace("'", " ")
    txt = txt.lower().split()
    txt = [word for word in txt if word.isalpha()] # очистка от символов и цифр
    txt = [lemmatizer.lemmatize(word) for word in txt] # лемматизация
    txt = [word for word in txt if word not in stopwords.words('english')] # очистка от стопслов
    return ' '.join(txt)

df_train['tweet'] = df_train['tweet'].apply(preprocess_text)
df_val['tweet'] = df_val['tweet'].apply(preprocess_text)
df_test['tweet'] = df_test['tweet'].apply(preprocess_text)

In [210]:
df_train.head()

,label,tweet
id,,
28599,0,attending user user
27507,0,nimwit libtards embrace crookedhillary cronyga...
16242,0,still sad pray today boanoite triste instagram...
23285,0,user le month till york host one greatest annu...
19694,0,user user user person change way living listen...


In [211]:
train_corpus = ''.join(df_train['tweet'].values)

In [212]:
from nltk.tokenize import word_tokenize

In [213]:
tokens = word_tokenize(train_corpus)
tokens[:10]

['attending',
 'user',
 'usernimwit',
 'libtards',
 'embrace',
 'crookedhillary',
 'cronygarchy',
 'braincell',
 'anemicstill',
 'sad']

In [214]:
MAX_WORDS = 2000
MAX_LEN = 20

In [215]:
from nltk.probability import FreqDist

In [216]:
dist = FreqDist(tokens)
tokens_top = [items[0] for items in dist.most_common(MAX_WORDS - 1)]

tokens_top[:10]

['user', 'day', 'love', 'u', 'amp', 'like', 'life', 'happy', 'get', 'wa']

In [217]:
vocabulary = {word: count for count, word in dict(enumerate(tokens_top, 1)).items()}
vocabulary

{'user': 1,
 'day': 2,
 'love': 3,
 'u': 4,
 'amp': 5,
 'like': 6,
 'life': 7,
 'happy': 8,
 'get': 9,
 'wa': 10,
 'today': 11,
 'new': 12,
 'people': 13,
 'time': 14,
 'make': 15,
 'father': 16,
 'one': 17,
 'see': 18,
 'take': 19,
 'go': 20,
 'good': 21,
 'smile': 22,
 'work': 23,
 'friend': 24,
 'bihday': 25,
 'ha': 26,
 'need': 27,
 'bull': 28,
 'want': 29,
 'girl': 30,
 'positive': 31,
 'weekend': 32,
 'thankful': 33,
 'way': 34,
 'family': 35,
 'week': 36,
 'summer': 37,
 'useruser': 38,
 'year': 39,
 'got': 40,
 'great': 41,
 'thing': 42,
 'healthy': 43,
 'morning': 44,
 'home': 45,
 'fun': 46,
 'really': 47,
 'best': 48,
 'going': 49,
 'back': 50,
 'come': 51,
 'friday': 52,
 'first': 53,
 'know': 54,
 'blog': 55,
 'beautiful': 56,
 'world': 57,
 'feel': 58,
 'music': 59,
 'think': 60,
 'dad': 61,
 'cute': 62,
 'night': 63,
 'never': 64,
 'look': 65,
 'tomorrow': 66,
 'right': 67,
 'say': 68,
 'much': 69,
 'trump': 70,
 'wait': 71,
 'iam': 72,
 'silver': 73,
 'gold': 74,
 'sad'

In [218]:
# Переведём твиты в набор индексов, добавим паддинг

def text_to_sequence(txt, maxlen):
    result = []
    tokens = word_tokenize(txt)
    for word in tokens:
        if word in vocabulary:
            result.append(vocabulary[word])

    padding = [0] * (maxlen-len(result))
    return result[-maxlen:] + padding

In [219]:
X_train = np.array([text_to_sequence(txt, MAX_LEN) for txt in df_train['tweet'].values])
X_val = np.array([text_to_sequence(txt, MAX_LEN) for txt in df_val['tweet'].values])

X_train.shape, X_val.shape

((25569, 20), (6393, 20))

In [221]:
import torch.nn as nn

In [222]:
# Инициализируем свёрточную нейросеть
class Net(nn.Module):
    def __init__(self, vocab_size=2000, embedding_dim=128, out_channel=64, num_classes=1, threshold=0.5):
        super().__init__()
        self.threshold = threshold
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0) 
        self.conv_1 = nn.Conv1d(embedding_dim, out_channel, kernel_size=3, padding='same') 
        self.bn1 = nn.BatchNorm1d(out_channel)
        self.pool = nn.MaxPool1d(2)
        self.relu = nn.ReLU()
        self.linear_1 = nn.Linear(out_channel, num_classes)
        self.dp1d = nn.Dropout1d(0.5)
        self.dp = nn.Dropout(0.5)
        
        
    def forward(self, x):     # Для понимания обозначим размеры входных данных на каждом слое,
                              # используя гиперпараметры по умолчанию и max_len=20
        x = self.embedding(x) # (1, 20) -> (1, 20, 128)       
        x = x.permute(0, 2, 1) # (1, 20, 128) -> (1, 128, 20)
        x = self.conv_1(x) # (1, 128, 20) -> (1, 64, 20)
        x = self.bn1(x)
        x = self.dp1d(x)
        x = self.relu(x)
        x = self.pool(x) # (1, 64, 20) -> (1, 64, 10)
        
        x = torch.max(x, axis=2).values # (1, 64, 10) -> (1, 64)
        x = self.dp(x)
        x = self.linear_1(x) # (1, 64) -> (1, 1)
        x = torch.sigmoid(x)
        return x
    
    def predict(self, x):
        x = torch.IntTensor(x).to(device)
        x = self.forward(x)
        x = torch.squeeze((x > self.threshold).int())
        return x

In [223]:
!pip install torchinfo

Defaulting to user installation because normal site-packages is not writeable


In [224]:
from torchinfo import summary

In [225]:
import torch

In [226]:
summary(Net(), input_data=torch.IntTensor(X_train[np.newaxis, 0]))

Layer (type:depth-idx)                   Output Shape              Param #
Net                                      [1, 1]                    --
├─Embedding: 1-1                         [1, 20, 128]              256,000
├─Conv1d: 1-2                            [1, 64, 20]               24,640
├─BatchNorm1d: 1-3                       [1, 64, 20]               128
├─Dropout1d: 1-4                         [1, 64, 20]               --
├─ReLU: 1-5                              [1, 64, 20]               --
├─MaxPool1d: 1-6                         [1, 64, 10]               --
├─Dropout: 1-7                           [1, 64]                   --
├─Linear: 1-8                            [1, 1]                    65
Total params: 280,833
Trainable params: 280,833
Non-trainable params: 0
Total mult-adds (M): 0.75
Input size (MB): 0.00
Forward/backward pass size (MB): 0.04
Params size (MB): 1.12
Estimated Total Size (MB): 1.16

In [227]:
from torch.utils.data import DataLoader, Dataset

In [228]:
class DataWrapper(Dataset):
    def __init__(self, data, target):
        self.data = torch.from_numpy(data)
        self.target = torch.from_numpy(target)
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
            
        return x, y
    
    def __len__(self):
        return len(self.data)

In [229]:
torch.random.manual_seed(10)

train_dataset = DataWrapper(X_train, df_train['label'].values)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)

val_dataset = DataWrapper(X_val, df_val['label'].values)
val_loader = DataLoader(val_dataset, batch_size=512, shuffle=True)

In [230]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [232]:
def train_nn(epochs, embedding_dim, hidden_size, lr, threshold=0.5, return_model=False):
# def train_nn(epochs, embedding_dim, hidden_size=32, lr=0.001, threshold=0.5, return_model=False):
    
    torch.random.manual_seed(10)
    torch.backends.cudnn.deterministic = True

    net = Net(vocab_size=MAX_WORDS, embedding_dim=embedding_dim, 
              out_channel=hidden_size, threshold=threshold).to(device)
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.BCELoss()

    for epoch in range(epochs):
        train_losses = np.array([])
        test_losses = np.array([])
        tp, fp, tn, fn = 0, 0, 0, 0

        for i, (inputs, labels) in enumerate(train_loader):
            net.train()
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = net(inputs)

            loss = criterion(outputs, labels.float().view(-1, 1))
            loss.backward()
            optimizer.step()

            train_losses = np.append(train_losses, loss.item())

            net.eval()
            outputs = torch.squeeze((net(inputs) > threshold).int())

            tp += ((labels == 1) & (outputs == 1)).sum().item()
            tn += ((labels == 0) & (outputs == 0)).sum().item()
            fp += ((labels == 0) & (outputs == 1)).sum().item()
            fn += ((labels == 1) & (outputs == 0)).sum().item()

        precision = tp / (tp + fp) if (tp + fp) != 0 else 0
        recall = tp / (tp + fn) if (tp + fn) != 0 else 0

        f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) != 0 else 0

        print(f'Epoch [{epoch + 1}/{epochs}]. ' \
              f'Loss: {train_losses.mean():.3f}. ' \
              f'F1-score: {f1_score:.3f}', end='. ')

        tp, fp, tn, fn = 0, 0, 0, 0

        with torch.no_grad():
            for i, (inputs, labels) in enumerate(val_loader):

                inputs, labels = inputs.to(device), labels.to(device)
                outputs = net(inputs)

                loss = criterion(outputs, labels.float().view(-1, 1))
                test_losses = np.append(test_losses, loss.item())

                tp += ((labels == 1) & (torch.squeeze((outputs > threshold).int()) == 1)).sum()
                tn += ((labels == 0) & (torch.squeeze((outputs > threshold).int()) == 0)).sum()
                fp += ((labels == 0) & (torch.squeeze((outputs > threshold).int()) == 1)).sum()
                fn += ((labels == 1) & (torch.squeeze((outputs > threshold).int()) == 0)).sum()

        precision = tp / (tp + fp) if (tp + fp) != 0 else 0
        recall = tp / (tp + fn) if (tp + fn) != 0 else 0

        f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) != 0 else 0

        print(f'Test loss: {test_losses.mean():.3f}. Test F1-score: {f1_score:.3f}. Precision: {precision:.3f}. Recall: {recall:.3f}')

    print('Training is finished!')
    if return_model:
        return net

In [233]:
train_nn(epochs=10, embedding_dim=64, hidden_size=16, lr=0.001)

Epoch [1/10]. Loss: 0.892. F1-score: 0.096. Test loss: 0.270. Test F1-score: 0.026. Precision: 0.076. Recall: 0.016
Epoch [2/10]. Loss: 0.504. F1-score: 0.014. Test loss: 0.249. Test F1-score: 0.012. Precision: 0.042. Recall: 0.007
Epoch [3/10]. Loss: 0.432. F1-score: 0.019. Test loss: 0.237. Test F1-score: 0.023. Precision: 0.081. Recall: 0.013
Epoch [4/10]. Loss: 0.383. F1-score: 0.029. Test loss: 0.230. Test F1-score: 0.018. Precision: 1.000. Recall: 0.009
Epoch [5/10]. Loss: 0.353. F1-score: 0.030. Test loss: 0.225. Test F1-score: 0.018. Precision: 1.000. Recall: 0.009
Epoch [6/10]. Loss: 0.332. F1-score: 0.032. Test loss: 0.222. Test F1-score: 0.018. Precision: 1.000. Recall: 0.009
Epoch [7/10]. Loss: 0.313. F1-score: 0.052. Test loss: 0.222. Test F1-score: 0.048. Precision: 1.000. Recall: 0.025
Epoch [8/10]. Loss: 0.298. F1-score: 0.063. Test loss: 0.216. Test F1-score: 0.018. Precision: 1.000. Recall: 0.009
Epoch [9/10]. Loss: 0.292. F1-score: 0.054. Test loss: 0.218. Test F1-sc

In [234]:
net_1=train_nn(epochs=50, embedding_dim=64, hidden_size=16, lr=0.001)

Epoch [1/50]. Loss: 0.892. F1-score: 0.096. Test loss: 0.270. Test F1-score: 0.026. Precision: 0.076. Recall: 0.016
Epoch [2/50]. Loss: 0.504. F1-score: 0.014. Test loss: 0.249. Test F1-score: 0.012. Precision: 0.042. Recall: 0.007
Epoch [3/50]. Loss: 0.432. F1-score: 0.019. Test loss: 0.237. Test F1-score: 0.023. Precision: 0.081. Recall: 0.013
Epoch [4/50]. Loss: 0.383. F1-score: 0.029. Test loss: 0.230. Test F1-score: 0.018. Precision: 1.000. Recall: 0.009
Epoch [5/50]. Loss: 0.353. F1-score: 0.030. Test loss: 0.225. Test F1-score: 0.018. Precision: 1.000. Recall: 0.009
Epoch [6/50]. Loss: 0.332. F1-score: 0.032. Test loss: 0.222. Test F1-score: 0.018. Precision: 1.000. Recall: 0.009
Epoch [7/50]. Loss: 0.313. F1-score: 0.052. Test loss: 0.222. Test F1-score: 0.048. Precision: 1.000. Recall: 0.025
Epoch [8/50]. Loss: 0.298. F1-score: 0.063. Test loss: 0.216. Test F1-score: 0.018. Precision: 1.000. Recall: 0.009
Epoch [9/50]. Loss: 0.292. F1-score: 0.054. Test loss: 0.218. Test F1-sc

In [235]:
net_2=train_nn(epochs=50, embedding_dim=128, hidden_size=16, lr=0.001)

Epoch [1/50]. Loss: 0.658. F1-score: 0.091. Test loss: 0.242. Test F1-score: 0.093. Precision: 0.815. Recall: 0.049
Epoch [2/50]. Loss: 0.393. F1-score: 0.121. Test loss: 0.223. Test F1-score: 0.129. Precision: 0.969. Recall: 0.069
Epoch [3/50]. Loss: 0.330. F1-score: 0.145. Test loss: 0.214. Test F1-score: 0.144. Precision: 0.946. Recall: 0.078
Epoch [4/50]. Loss: 0.301. F1-score: 0.156. Test loss: 0.207. Test F1-score: 0.144. Precision: 0.946. Recall: 0.078
Epoch [5/50]. Loss: 0.275. F1-score: 0.166. Test loss: 0.202. Test F1-score: 0.141. Precision: 0.971. Recall: 0.076
Epoch [6/50]. Loss: 0.264. F1-score: 0.175. Test loss: 0.199. Test F1-score: 0.160. Precision: 0.975. Recall: 0.087
Epoch [7/50]. Loss: 0.247. F1-score: 0.184. Test loss: 0.195. Test F1-score: 0.156. Precision: 0.974. Recall: 0.085
Epoch [8/50]. Loss: 0.238. F1-score: 0.192. Test loss: 0.191. Test F1-score: 0.182. Precision: 0.978. Recall: 0.100
Epoch [9/50]. Loss: 0.233. F1-score: 0.216. Test loss: 0.186. Test F1-sc

In [236]:
net_3=train_nn(epochs=50, embedding_dim=256, hidden_size=16, lr=0.001)

Epoch [1/50]. Loss: 0.758. F1-score: 0.153. Test loss: 0.242. Test F1-score: 0.090. Precision: 0.227. Recall: 0.056
Epoch [2/50]. Loss: 0.386. F1-score: 0.135. Test loss: 0.216. Test F1-score: 0.109. Precision: 0.897. Recall: 0.058
Epoch [3/50]. Loss: 0.339. F1-score: 0.185. Test loss: 0.206. Test F1-score: 0.136. Precision: 0.892. Recall: 0.074
Epoch [4/50]. Loss: 0.307. F1-score: 0.215. Test loss: 0.197. Test F1-score: 0.173. Precision: 0.896. Recall: 0.096
Epoch [5/50]. Loss: 0.288. F1-score: 0.223. Test loss: 0.189. Test F1-score: 0.181. Precision: 0.938. Recall: 0.100
Epoch [6/50]. Loss: 0.267. F1-score: 0.244. Test loss: 0.187. Test F1-score: 0.185. Precision: 0.939. Recall: 0.103
Epoch [7/50]. Loss: 0.256. F1-score: 0.255. Test loss: 0.184. Test F1-score: 0.202. Precision: 0.911. Recall: 0.114
Epoch [8/50]. Loss: 0.242. F1-score: 0.277. Test loss: 0.178. Test F1-score: 0.237. Precision: 0.924. Recall: 0.136
Epoch [9/50]. Loss: 0.231. F1-score: 0.301. Test loss: 0.177. Test F1-sc

In [238]:
net_4 = train_nn(epochs=50, embedding_dim=512, hidden_size=16, lr=0.001)

Epoch [1/50]. Loss: 0.517. F1-score: 0.155. Test loss: 0.224. Test F1-score: 0.136. Precision: 0.312. Recall: 0.087
Epoch [2/50]. Loss: 0.315. F1-score: 0.216. Test loss: 0.200. Test F1-score: 0.168. Precision: 0.387. Recall: 0.107
Epoch [3/50]. Loss: 0.277. F1-score: 0.256. Test loss: 0.191. Test F1-score: 0.197. Precision: 0.408. Recall: 0.129
Epoch [4/50]. Loss: 0.259. F1-score: 0.295. Test loss: 0.187. Test F1-score: 0.222. Precision: 0.452. Recall: 0.147
Epoch [5/50]. Loss: 0.243. F1-score: 0.324. Test loss: 0.176. Test F1-score: 0.287. Precision: 0.517. Recall: 0.199
Epoch [6/50]. Loss: 0.226. F1-score: 0.390. Test loss: 0.173. Test F1-score: 0.357. Precision: 0.856. Recall: 0.225
Epoch [7/50]. Loss: 0.215. F1-score: 0.425. Test loss: 0.170. Test F1-score: 0.368. Precision: 0.861. Recall: 0.234
Epoch [8/50]. Loss: 0.208. F1-score: 0.468. Test loss: 0.168. Test F1-score: 0.378. Precision: 0.852. Recall: 0.243
Epoch [9/50]. Loss: 0.202. F1-score: 0.471. Test loss: 0.165. Test F1-sc

In [239]:
net_5 = train_nn(epochs=50, embedding_dim=1024, hidden_size=16, lr=0.001)

Epoch [1/50]. Loss: 0.444. F1-score: 0.177. Test loss: 0.202. Test F1-score: 0.195. Precision: 0.441. Recall: 0.125
Epoch [2/50]. Loss: 0.300. F1-score: 0.237. Test loss: 0.190. Test F1-score: 0.230. Precision: 0.908. Recall: 0.132
Epoch [3/50]. Loss: 0.272. F1-score: 0.290. Test loss: 0.184. Test F1-score: 0.238. Precision: 0.938. Recall: 0.136
Epoch [4/50]. Loss: 0.253. F1-score: 0.317. Test loss: 0.177. Test F1-score: 0.284. Precision: 0.926. Recall: 0.167
Epoch [5/50]. Loss: 0.240. F1-score: 0.355. Test loss: 0.172. Test F1-score: 0.324. Precision: 0.926. Recall: 0.196
Epoch [6/50]. Loss: 0.225. F1-score: 0.383. Test loss: 0.170. Test F1-score: 0.367. Precision: 0.912. Recall: 0.230
Epoch [7/50]. Loss: 0.220. F1-score: 0.426. Test loss: 0.165. Test F1-score: 0.404. Precision: 0.943. Recall: 0.257
Epoch [8/50]. Loss: 0.204. F1-score: 0.464. Test loss: 0.164. Test F1-score: 0.445. Precision: 0.910. Recall: 0.295
Epoch [9/50]. Loss: 0.193. F1-score: 0.502. Test loss: 0.159. Test F1-sc

In [240]:
net_6 = train_nn(epochs=50, embedding_dim=512, hidden_size=32, lr=0.001)

Epoch [1/50]. Loss: 0.498. F1-score: 0.189. Test loss: 0.217. Test F1-score: 0.113. Precision: 0.964. Recall: 0.060
Epoch [2/50]. Loss: 0.284. F1-score: 0.198. Test loss: 0.196. Test F1-score: 0.174. Precision: 0.935. Recall: 0.096
Epoch [3/50]. Loss: 0.242. F1-score: 0.250. Test loss: 0.188. Test F1-score: 0.220. Precision: 0.903. Recall: 0.125
Epoch [4/50]. Loss: 0.223. F1-score: 0.291. Test loss: 0.180. Test F1-score: 0.273. Precision: 0.900. Recall: 0.161
Epoch [5/50]. Loss: 0.205. F1-score: 0.342. Test loss: 0.171. Test F1-score: 0.328. Precision: 0.891. Recall: 0.201
Epoch [6/50]. Loss: 0.192. F1-score: 0.403. Test loss: 0.167. Test F1-score: 0.375. Precision: 0.898. Recall: 0.237
Epoch [7/50]. Loss: 0.180. F1-score: 0.453. Test loss: 0.162. Test F1-score: 0.393. Precision: 0.890. Recall: 0.252
Epoch [8/50]. Loss: 0.169. F1-score: 0.502. Test loss: 0.158. Test F1-score: 0.436. Precision: 0.872. Recall: 0.290
Epoch [9/50]. Loss: 0.162. F1-score: 0.549. Test loss: 0.156. Test F1-sc

In [241]:
net_7 = train_nn(epochs=50, embedding_dim=512, hidden_size=64, lr=0.001)

Epoch [1/50]. Loss: 0.295. F1-score: 0.193. Test loss: 0.191. Test F1-score: 0.252. Precision: 0.868. Recall: 0.147
Epoch [2/50]. Loss: 0.204. F1-score: 0.346. Test loss: 0.170. Test F1-score: 0.375. Precision: 0.877. Recall: 0.239
Epoch [3/50]. Loss: 0.179. F1-score: 0.455. Test loss: 0.161. Test F1-score: 0.408. Precision: 0.875. Recall: 0.266
Epoch [4/50]. Loss: 0.165. F1-score: 0.520. Test loss: 0.153. Test F1-score: 0.482. Precision: 0.857. Recall: 0.335
Epoch [5/50]. Loss: 0.147. F1-score: 0.597. Test loss: 0.146. Test F1-score: 0.523. Precision: 0.819. Recall: 0.384
Epoch [6/50]. Loss: 0.139. F1-score: 0.636. Test loss: 0.145. Test F1-score: 0.538. Precision: 0.821. Recall: 0.400
Epoch [7/50]. Loss: 0.128. F1-score: 0.675. Test loss: 0.148. Test F1-score: 0.556. Precision: 0.849. Recall: 0.413
Epoch [8/50]. Loss: 0.124. F1-score: 0.704. Test loss: 0.146. Test F1-score: 0.573. Precision: 0.837. Recall: 0.435
Epoch [9/50]. Loss: 0.114. F1-score: 0.727. Test loss: 0.145. Test F1-sc

In [242]:
net_8 = train_nn(epochs=50, embedding_dim=512, hidden_size=128, lr=0.001)

Epoch [1/50]. Loss: 0.329. F1-score: 0.225. Test loss: 0.183. Test F1-score: 0.314. Precision: 0.544. Recall: 0.221
Epoch [2/50]. Loss: 0.203. F1-score: 0.406. Test loss: 0.164. Test F1-score: 0.438. Precision: 0.831. Recall: 0.297
Epoch [3/50]. Loss: 0.174. F1-score: 0.511. Test loss: 0.155. Test F1-score: 0.488. Precision: 0.829. Recall: 0.346
Epoch [4/50]. Loss: 0.154. F1-score: 0.580. Test loss: 0.150. Test F1-score: 0.513. Precision: 0.859. Recall: 0.366
Epoch [5/50]. Loss: 0.138. F1-score: 0.638. Test loss: 0.146. Test F1-score: 0.565. Precision: 0.828. Recall: 0.429
Epoch [6/50]. Loss: 0.126. F1-score: 0.685. Test loss: 0.142. Test F1-score: 0.573. Precision: 0.821. Recall: 0.440
Epoch [7/50]. Loss: 0.117. F1-score: 0.719. Test loss: 0.144. Test F1-score: 0.577. Precision: 0.816. Recall: 0.446
Epoch [8/50]. Loss: 0.107. F1-score: 0.757. Test loss: 0.143. Test F1-score: 0.599. Precision: 0.836. Recall: 0.467
Epoch [9/50]. Loss: 0.102. F1-score: 0.772. Test loss: 0.141. Test F1-sc

In [243]:
net_9 = train_nn(epochs=50, embedding_dim=512, hidden_size=256, lr=0.001)

Epoch [1/50]. Loss: 0.379. F1-score: 0.277. Test loss: 0.179. Test F1-score: 0.421. Precision: 0.588. Recall: 0.328
Epoch [2/50]. Loss: 0.200. F1-score: 0.449. Test loss: 0.165. Test F1-score: 0.436. Precision: 0.616. Recall: 0.337
Epoch [3/50]. Loss: 0.161. F1-score: 0.595. Test loss: 0.152. Test F1-score: 0.536. Precision: 0.831. Recall: 0.395
Epoch [4/50]. Loss: 0.137. F1-score: 0.649. Test loss: 0.144. Test F1-score: 0.562. Precision: 0.823. Recall: 0.426
Epoch [5/50]. Loss: 0.124. F1-score: 0.706. Test loss: 0.142. Test F1-score: 0.563. Precision: 0.848. Recall: 0.422
Epoch [6/50]. Loss: 0.113. F1-score: 0.749. Test loss: 0.140. Test F1-score: 0.603. Precision: 0.853. Recall: 0.467
Epoch [7/50]. Loss: 0.103. F1-score: 0.772. Test loss: 0.141. Test F1-score: 0.598. Precision: 0.855. Recall: 0.460
Epoch [8/50]. Loss: 0.094. F1-score: 0.813. Test loss: 0.139. Test F1-score: 0.612. Precision: 0.826. Recall: 0.487
Epoch [9/50]. Loss: 0.087. F1-score: 0.828. Test loss: 0.145. Test F1-sc

In [248]:
net_10 = train_nn(epochs=50, embedding_dim=512, hidden_size=256, lr=0.01, return_model=True)

Epoch [1/50]. Loss: 0.378. F1-score: 0.281. Test loss: 0.166. Test F1-score: 0.419. Precision: 0.885. Recall: 0.275
Epoch [2/50]. Loss: 0.151. F1-score: 0.558. Test loss: 0.148. Test F1-score: 0.561. Precision: 0.781. Recall: 0.438
Epoch [3/50]. Loss: 0.124. F1-score: 0.688. Test loss: 0.149. Test F1-score: 0.583. Precision: 0.795. Recall: 0.460
Epoch [4/50]. Loss: 0.104. F1-score: 0.741. Test loss: 0.150. Test F1-score: 0.604. Precision: 0.762. Recall: 0.500
Epoch [5/50]. Loss: 0.096. F1-score: 0.772. Test loss: 0.156. Test F1-score: 0.598. Precision: 0.753. Recall: 0.496
Epoch [6/50]. Loss: 0.088. F1-score: 0.816. Test loss: 0.167. Test F1-score: 0.611. Precision: 0.700. Recall: 0.542
Epoch [7/50]. Loss: 0.077. F1-score: 0.836. Test loss: 0.173. Test F1-score: 0.629. Precision: 0.702. Recall: 0.569
Epoch [8/50]. Loss: 0.069. F1-score: 0.865. Test loss: 0.196. Test F1-score: 0.608. Precision: 0.771. Recall: 0.502
Epoch [9/50]. Loss: 0.063. F1-score: 0.884. Test loss: 0.202. Test F1-sc

### Вывод: Epoch>8 приводит к переобучению и снижению метрики F1-score на тестовых данных, embedding_dim=512 при прочих равных условиях показало наилучшую  метрику F1-score, hidden_size=256 при соблюдении соотношении 1:2 с embedding_dim также улучшает показатель F1-score (сильна корреляция с embedding_dim), lr=0.01 довел точность до максимума (lr=0.1 просто "сломал" работу нейронной сети). Повысить точность нейронной сети помогла тонкая настройка основных параметров Epoch, embedding_dim и hidden_size. Лучший результат Test F1-score=0.629 на 7 эпохе с параметрами (embedding_dim=512, hidden_size=256, lr=0.01). 